In [1]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: LIBS
# Previous cells: none
# Next cells: ALL
# --------------------------------------------------------------------------------------------------------------

import numpy as np
import math
#from playsound import playsound

In [4]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: DIR
# Previous cells: none
# Next cells: ALL
# --------------------------------------------------------------------------------------------------------------

import sys
import os
os.chdir('E:\JUPYTER_files')
os.path.dirname(sys.executable)

'C:\\Users\\Stargazer\\Anaconda3\\envs\\safeclone'

In [5]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: FUNCTIONS
# Previous cells: LIBS
# Next cells: ALL
# --------------------------------------------------------------------------------------------------------------

# This is a bubble sort... I think...
def mySort(arr,index,secondary):
    for i in range(len(arr)):        
        for i2 in range(i,0,-1):
            if int(arr[i2][index]) < int(arr[i2-1][index]):
                arr[i2], arr[i2-1] = arr[i2-1], arr[i2]    
            elif secondary >= 0 and int(arr[i2][index]) == int(arr[i2-1][index]) and int(arr[i2][secondary]) < int(arr[i2-1][secondary]):
                arr[i2], arr[i2-1] = arr[i2-1], arr[i2]

def my2DSort(arr,index):
    for i in range(len(arr)-1):
        for i2 in range(len(arr)-i-1):
            if int(arr[i2][index]) > int(arr[i2+1][index]):
                arr[i2], arr[i2+1] = arr[i2+1], arr[i2]

# This is a binary search... I guess... FOR SORTED LISTS!
def mySearch(arr,index,x):
    l = 0
    h = len(arr)
    m = 1
    while(True):
        m = int((h+l)/2)
        if int(x) == int(arr[m][index]):
            return arr[m]
        if m == l or m == h:
            break
        if int(x) > int(arr[m][index]):
            l = m
        else:
            h = m
    return []

# this appends an item to a list if the item is not already in the list
def insert_to_list_unique(list,x):
    if(x not in list):
        list.append(x)

def myUnsortedSearch(arr,index,x):
    appearance_list = []
    for item in arr:
        if(item[index] == x):
            appearance_list.append(item)
    return appearance_list

# counts how many times and item apprears in a list
def appearance_counter(arr,index,x):
    cnt = 0
    for item in arr:
        if(item[index] == x):
            cnt += 1
    return cnt

# x = 0 for year, 1 for month, 0 for day
def dateStrip(date):
    return date.split(' ')[0].split('-')[0] + date.split(' ')[0].split('-')[1] + date.split(' ')[0].split('-')[2]

# Considering average month is 30.4 days -> let's say 30.
# Ages above 89 are obscured into the over 300 range. Disregard these values as said by PHYSIONET.
def days_diff(date1,date2):
    d1,d2 = [],[]
    for x in date1.split(' ')[0].split('-'):
        d1.append(int(x))
    for x in date2.split(' ')[0].split('-'):
        d2.append(int(x))
    diff = [d1[0]-d2[0],d1[1]-d2[1],d1[2]-d2[2]]
    stay = diff[2] + diff[1]*30 + diff[0]*365
    if stay < 0:
        stay = -stay
    return stay
        
def years_diff(date1,date2):
    d1,d2 = [],[]
    for x in date1.split(' ')[0].split('-'):
        d1.append(int(x))
    for x in date2.split(' ')[0].split('-'):
        d2.append(int(x))
    age = d1[0]-d2[0]
    if age < 0:
        age = -age
    if age < 90:
        return age
    else:
        return -1

# Splits ICD9 codes into their respective categories, as done in the studied paper
def categorize(x):
    if 'V' in x:
        return 'V01 - 99'
    if 'E' in x:
        return 'E001 - E999'
    a = "not"
    b = "applicable"
    points = [1,140,240,280,290,320,390,460,520,580,630,680,710,740,760,780,800,1000]
    temp = int(x)
    if temp > 999:
        temp /= 10
    for i in range(len(points)-1):
        if(temp >= points[i] and temp < points[i+1]):
            a = str(points[i])
            b = str(points[i+1]-1)
    return a + " - " + b

# Strips ICD9 codes as per the 3-digit data aggregation (NOTE: E-codes don't have dot(.) subcategories) (that note is a lie)
def digitize3(x):
    if 'E' in x:
        return x[:4]
    else:
        return x[:3]

# Used to sort the ICDs from lowest to highest
def icdSort(l):
    templist = l.copy()
    finalist = []
    while(templist != []):
        toappend = templist[0]
        for x in templist[1:]:
            if 'V' in x:
                if 'V' not in toappend:
                    toappend = x
                elif int(x[1:]) < int(toappend[1:]):
                    toappend = x
            elif 'E' in x:
                if 'V' not in toappend:
                    if 'E' not in toappend:
                        toappend = x
                    elif int(x[1:]) < int(toappend[1:]):
                        toappend = x
            elif 'V' not in toappend and 'E' not in toappend and int(x) < int(toappend):
                    toappend = x
        finalist.append(toappend)
        templist.remove(toappend)
    return finalist

In [6]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ADMISSIONS
# Previous cells: LIBS
# Next cells: VCOUNTERS
# --------------------------------------------------------------------------------------------------------------

# Reads the ADMISSIONS.csv file and assigns the data into a 2D list, filtered to only contain the needed data.
temp = open('ADMISSIONS.csv')
admissions, visits, singular = [],[],[]
for line in temp:
    singular = list(line[:-1].replace('"', "").split(','))
    admissions.append([singular[1],singular[2],singular[3],singular[4],singular[5],singular[12],singular[13]])
    visits.append([singular[1],singular[2]])
admissions[:5]

[['SUBJECT_ID',
  'HADM_ID',
  'ADMITTIME',
  'DISCHTIME',
  'DEATHTIME',
  'MARITAL_STATUS',
  'ETHNICITY'],
 ['22',
  '165315',
  '2196-04-09 12:26:00',
  '2196-04-10 15:54:00',
  '',
  'MARRIED',
  'WHITE'],
 ['23',
  '152223',
  '2153-09-03 07:15:00',
  '2153-09-08 19:10:00',
  '',
  'MARRIED',
  'WHITE'],
 ['23',
  '124321',
  '2157-10-18 19:34:00',
  '2157-10-25 14:00:00',
  '',
  'MARRIED',
  'WHITE'],
 ['24',
  '161859',
  '2139-06-06 16:14:00',
  '2139-06-09 12:48:00',
  '',
  'SINGLE',
  'WHITE']]

In [7]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: VCOUNTERS
# Previous cells: ADMISSIONS
# Next cells: OVER1COUNTERS
# --------------------------------------------------------------------------------------------------------------

# creates a list with a visit counter per patient
vcounters = []
for x in visits:
    flag = True
    for i in range(len(vcounters)):
        if(x[0] == vcounters[-i-1][0]):
            vcounters[-i-1][1] += 1
            flag = False
            break
    if(flag):
        vcounters.append([x[0],1])
print("Number of patients: %s" % (len(vcounters)))

Number of patients: 46521


In [8]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: OVER1COUNTERS
# Previous cells: VCOUNTERS
# Next cells: ADMISSIONS_FILTERED, PATIENTS_FILTERED
# --------------------------------------------------------------------------------------------------------------

# An array with the patient_IDs of patients who visited more than once. Length signifies the amount of these patients
over1counters = []
for x in vcounters:
    if x[1] > 1:
        over1counters.append(x)
over1counters = np.array(over1counters)
len(over1counters)

7537

In [9]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ADMISSIONS_FILTERED
# Previous cells: OVER1COUNTERS
# Next cells: ADMISSION_SORT, ETHNICITIES_AND_MARITAL
# --------------------------------------------------------------------------------------------------------------

# Filters the admissions list, keeping only patients with 2 or more visits.
# [subject_id, hadm_id, admittime, dischtime, deathtime, marital_status, ethnicity]
admissions_filtered = []
for x in admissions:
    if(x[0] in over1counters[:,0]):
        admissions_filtered.append(x)
print("Shows admission and discharge date, marital status and ethnicity of each patient.")
print("Deathtime determines if the calculations continue.")
admissions_filtered[:5]

Shows admission and discharge date, marital status and ethnicity of each patient.
Deathtime determines if the calculations continue.


[['23',
  '152223',
  '2153-09-03 07:15:00',
  '2153-09-08 19:10:00',
  '',
  'MARRIED',
  'WHITE'],
 ['23',
  '124321',
  '2157-10-18 19:34:00',
  '2157-10-25 14:00:00',
  '',
  'MARRIED',
  'WHITE'],
 ['34',
  '115799',
  '2186-07-18 16:46:00',
  '2186-07-20 16:00:00',
  '',
  'MARRIED',
  'WHITE'],
 ['34',
  '144319',
  '2191-02-23 05:23:00',
  '2191-02-25 20:20:00',
  '',
  'MARRIED',
  'WHITE'],
 ['36',
  '182104',
  '2131-04-30 07:15:00',
  '2131-05-08 14:00:00',
  '',
  'MARRIED',
  'WHITE']]

In [10]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: DIAGNOSES
# Previous cells: LIBS
# Next cells: ICDS
# --------------------------------------------------------------------------------------------------------------

# Reads DIAGNOSES_ICD.csv and keeps a 3-collumn array, containing only  SUBJECT_ID, HAMD_ID and ICD9_CODE
diagnoses = open('DIAGNOSES_ICD.csv')
icds = []
singular = []
for line in diagnoses:
    singular = list(line[:-1].replace('"', "").split(','))
    icds.append([singular[1],singular[2],singular[4]])
icds[:5]

[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE'],
 ['109', '172335', '40301'],
 ['109', '172335', '486'],
 ['109', '172335', '58281'],
 ['109', '172335', '5855']]

In [11]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: PATIENTS
# Previous cells: LIBS
# Next cells: PATIENTS_FILTERED
# --------------------------------------------------------------------------------------------------------------

# Reads the PATIENTS.csv file and assigns the data to a 2D array.
temp = open('PATIENTS.csv')
patients = []
singular = []
for line in temp:
    singular = list(line[:-1].replace('"', "").split(','))
    patients.append([singular[1],singular[2],singular[3]])
patients[:5]

[['SUBJECT_ID', 'GENDER', 'DOB'],
 ['249', 'F', '2075-03-13 00:00:00'],
 ['250', 'F', '2164-12-27 00:00:00'],
 ['251', 'M', '2090-03-15 00:00:00'],
 ['252', 'M', '2078-03-06 00:00:00']]

In [12]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: PATIENTS_FILTERED
# Previous cells: OVER1COUNTERS, PATIENTS
# Next cells: PATIENT_SORT
# --------------------------------------------------------------------------------------------------------------

# Filters the patients list, keeping only patients with 2 or more visits. [subject_id, gender, date_of_birth]
patients_filtered = []
for x in patients:
    if(x[0] in over1counters[:,0]):
        patients_filtered.append(x)
patients_filtered 

[['249', 'F', '2075-03-13 00:00:00'],
 ['252', 'M', '2078-03-06 00:00:00'],
 ['256', 'M', '2086-07-31 00:00:00'],
 ['665', 'M', '2052-05-20 00:00:00'],
 ['674', 'F', '2113-12-14 00:00:00'],
 ['679', 'F', '2059-11-04 00:00:00'],
 ['689', 'F', '2128-12-27 00:00:00'],
 ['690', 'M', '2109-09-24 00:00:00'],
 ['695', 'F', '2093-05-14 00:00:00'],
 ['698', 'F', '1864-11-16 00:00:00'],
 ['703', 'M', '2032-03-08 00:00:00'],
 ['711', 'M', '2100-03-06 00:00:00'],
 ['720', 'F', '2108-08-25 00:00:00'],
 ['721', 'M', '2075-02-23 00:00:00'],
 ['725', 'M', '2056-08-06 00:00:00'],
 ['728', 'M', '2070-08-14 00:00:00'],
 ['731', 'F', '2073-11-29 00:00:00'],
 ['735', 'F', '2068-03-06 00:00:00'],
 ['745', 'M', '2142-06-14 00:00:00'],
 ['747', 'M', '2063-03-10 00:00:00'],
 ['755', 'F', '2127-04-25 00:00:00'],
 ['757', 'F', '2137-08-13 00:00:00'],
 ['771', 'M', '2095-12-27 00:00:00'],
 ['773', 'M', '2051-06-10 00:00:00'],
 ['776', 'M', '2123-02-22 00:00:00'],
 ['779', 'M', '2077-03-13 00:00:00'],
 ['781', 'F'

In [13]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: PATIENT_SORT
# Previous cells: PATIENTS_FILTERED, FUNCTIONS
# Next cells: FINAL_DATA_RAW
# --------------------------------------------------------------------------------------------------------------

# sorted by SUBJECT_ID

mySort(patients_filtered,0,-1)

In [14]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ADMISSION_SORT
# Previous cells: ADMISSIONS_FILTERED, FUNCTIONS
# Next cells: FINAL_DATA_RAW
# --------------------------------------------------------------------------------------------------------------

# sorted by HADM_ID

mySort(admissions_filtered,1,-1)

In [15]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: FINAL_DATA_RAW
# Previous cells: PATIENT_SORT, ADMISSION_SORT
# Next cells: FINAL_DATA
# --------------------------------------------------------------------------------------------------------------

# Contains all the unrpocessed data we'll need for the final array, minus the ICDs.
# [subject_id, hadm_id, admittime, dischtime, deathtime, marital_status, ethnicity, gender, date_of_birth]
final_data_raw = []
for i in range(len(admissions_filtered)):
    singular = []
    singular.append(admissions_filtered[i][0])
    singular.append(admissions_filtered[i][1])
    singular.append(admissions_filtered[i][2])
    singular.append(admissions_filtered[i][3])
    singular.append(admissions_filtered[i][4])
    singular.append(admissions_filtered[i][5])
    singular.append(admissions_filtered[i][6])
    temp = mySearch(patients_filtered,0,singular[0])
    singular.append(temp[1])
    singular.append(temp[2])
    final_data_raw.append(singular)
final_data_raw

[['58526',
  '100001',
  '2117-09-11 11:46:00',
  '2117-09-17 16:45:00',
  '',
  'DIVORCED',
  'WHITE',
  'F',
  '2082-03-21 00:00:00'],
 ['54610',
  '100003',
  '2150-04-17 15:34:00',
  '2150-04-21 17:30:00',
  '',
  'SINGLE',
  'WHITE',
  'M',
  '2090-05-19 00:00:00'],
 ['9895',
  '100006',
  '2108-04-06 15:49:00',
  '2108-04-18 17:18:00',
  '',
  'SINGLE',
  'BLACK/AFRICAN AMERICAN',
  'F',
  '2059-05-07 00:00:00'],
 ['23804',
  '100014',
  '2111-03-05 03:42:00',
  '2111-03-05 19:00:00',
  '',
  'SINGLE',
  'WHITE',
  'F',
  '2061-12-29 00:00:00'],
 ['58128',
  '100018',
  '2176-08-29 10:30:00',
  '2176-09-06 17:13:00',
  '',
  'MARRIED',
  'WHITE',
  'M',
  '2121-02-13 00:00:00'],
 ['67987',
  '100019',
  '2138-06-09 07:15:00',
  '2138-06-12 11:16:00',
  '',
  'MARRIED',
  'WHITE',
  'M',
  '2111-02-28 00:00:00'],
 ['9973',
  '100020',
  '2142-11-30 01:53:00',
  '2142-12-10 16:32:00',
  '',
  'MARRIED',
  'WHITE',
  'M',
  '2084-05-02 00:00:00'],
 ['29971',
  '100021',
  '2109-08-1

In [16]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: FINAL_DATA
# Previous cells: FINAL_DATA_RAW
# Next cells: ALL
# --------------------------------------------------------------------------------------------------------------

# Creates the final_data list with calculated ages, still lacks the ICDs.
# [subject_id, hadm_id, admittime, gender, age, ethnicity, marital_status, flag_true_if_survived, length_of_stay, ICD_list]

cleaning_id_list = []
final_data = []
for i in range(len(final_data_raw)):
    temp = []
    temp.append(final_data_raw[i][0])
    temp.append(final_data_raw[i][1])
    temp.append(dateStrip(final_data_raw[i][2]))
    temp.append(final_data_raw[i][7])
    temp.append(years_diff(final_data_raw[i][2],final_data_raw[i][8]))
    temp.append(final_data_raw[i][6])
    temp.append(final_data_raw[i][5])
    temp.append(len(final_data_raw[i][4]) == 0)
    temp.append(days_diff(final_data_raw[i][2],final_data_raw[i][3]))
    temp.append([])
    if int(temp[4]) > 0: # filters out patients with negative age due to error
        final_data.append(temp)
    else:
        insert_to_list_unique(cleaning_id_list,temp[0])
final_data[:5]

[['58526', '100001', '21170911', 'F', 35, 'WHITE', 'DIVORCED', True, 6, []],
 ['54610', '100003', '21500417', 'M', 60, 'WHITE', 'SINGLE', True, 4, []],
 ['9895',
  '100006',
  '21080406',
  'F',
  49,
  'BLACK/AFRICAN AMERICAN',
  'SINGLE',
  True,
  12,
  []],
 ['23804', '100014', '21110305', 'F', 50, 'WHITE', 'SINGLE', True, 0, []],
 ['58128', '100018', '21760829', 'M', 55, 'WHITE', 'MARRIED', True, 7, []]]

In [17]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ICDS
# Previous cells: FINAL_DATA
# Next cells: CLEANING
# --------------------------------------------------------------------------------------------------------------

# Adds the ICD codes to the final_data lists.
for x in icds:
    if x[0] in over1counters[:,0]:
        temp = mySearch(final_data,1,x[1])
        if temp != []:
            if 'Ε' in x[2]:
                temp[9].append(x[2][:5])
            else:
                temp[9].append(x[2][:4])

In [19]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: CLEANING
# Previous cells: ICDS
# Next cells: CLEANING_2ND_PASS
# --------------------------------------------------------------------------------------------------------------

for x in final_data:
    if x[9] == ['']:
        if(x[0] not in cleaning_id_list):
            cleaning_id_list.append(x[0])
        final_data.remove(x)
len(final_data)

18485

In [20]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: CLEANING_2ND_PASS
# Previous cells: CLEANING
# Next cells: FINAL_DATA_SORT
# --------------------------------------------------------------------------------------------------------------

for x in final_data:
    if(x[0] in cleaning_id_list):
        final_data.remove(x)
len(final_data)

18412

In [21]:
final_patients = []
for x in final_data:
    insert_to_list_unique(final_patients,x[0])
print("Before filtering:", len(over1counters), " patients.")
print("After filtering: ", len(final_patients), " patients.")

Before filtering: 7537  patients.
After filtering:  6871  patients.


In [22]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: FINAL_DATA_SORTED
# Previous cells: CLEANING
# Next cells: MEAN_AND_VARIANCE
# --------------------------------------------------------------------------------------------------------------

# Sorts final_data, first by SUBJECT_ID, then by admission date
mySort(final_data,0,2)

In [23]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: MEAN_AND_VARIANCE
# Previous cells: FINAL_DATA_SORTED
# Next cells: ETHNICITIES_AND_MARITAL
# --------------------------------------------------------------------------------------------------------------

# creates mean and variance normalization values for age and length of stay variables, it is applied later
mean_age, mean_los = 0, 0
for x in final_data:
    mean_age += x[4]
    mean_los += x[8]
mean_age /= len(final_data)
mean_los /= len(final_data)

variance_age, variance_los = 0, 0
for x in final_data:
    variance_age += (x[4]-mean_age)**2
    variance_los += (x[8]-mean_los)**2
variance_age /= len(final_data)-1
variance_age = math.sqrt(variance_age)
variance_los /= len(final_data)-1
variance_los = math.sqrt(variance_los)


In [24]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ETHNICITIES_AND_MARITAL
# Previous cells: ADMISSIONS_FILTERED
# Next cells: ETHNICITIES_SUPER, MARITAL_SUPER
# --------------------------------------------------------------------------------------------------------------

# creates the ethnicities and marital stati lists
all_ethnicities = []
all_marital_status = []
for x in admissions_filtered:
    if x[5]not in all_marital_status:
        all_marital_status.append(x[5])
    if x[6] not in all_ethnicities:
        all_ethnicities.append(x[6])

In [26]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ETHNICITIES_SUPER
# Previous cells: ETHNICITIES_AND_MARITAL
# Next cells: INSTANCE_LABEL_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

# unifies certain ethnicity one-hot values.. This, to be sure, is not thoroughly researched and even probably racist. If anyone is ever to tackle this again, do proper research. In my defence, the subject of DNA correlation to difference in disease prediction is abstract at best, plus the studied paper offered no insight on how they categorized ethnicities. I wash my hands of this.
all_ethnicities_super = []
all_ethnicities_super.append([ 'white - western', [ 'WHITE', 'PORTUGUESE', 'WHITE - OTHER EUROPEAN' ] ])
all_ethnicities_super.append([ 'black - african american', [ 'BLACK/AFRICAN AMERICAN', 'BLACK/HAITIAN', 'CARIBBEAN ISLAND' ] ])
all_ethnicities_super.append([ 'hispanic / latino', [ 'HISPANIC OR LATINO', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - SALVADORAN', 'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'HISPANIC/LATINO - COLOMBIAN', 'HISPANIC/LATINO - HONDURAN', 'WHITE - BRAZILIAN' ] ])
all_ethnicities_super.append([ 'white - eastern', [ 'WHITE - RUSSIAN', 'WHITE - EASTERN EUROPEAN' ] ])
all_ethnicities_super.append([ 'asian', [ 'ASIAN', 'ASIAN - OTHER', 'ASIAN - CHINESE', 'ASIAN - FILIPINO', 'ASIAN - CAMBODIAN', 'ASIAN - VIETNAMESE', 'ASIAN - THAI', 'ASIAN - KOREAN' ] ])
all_ethnicities_super.append([ 'black - non american', [ 'BLACK/CAPE VERDEAN', 'BLACK/AFRICAN' ] ])
all_ethnicities_super.append([ 'middle eastern', [ 'ASIAN - ASIAN INDIAN', 'MIDDLE EASTERN' ] ])
all_ethnicities_super.append([ 'multirace', [ 'MULTI RACE ETHNICITY' ] ])
all_ethnicities_super.append([ 'other / unknown / not specified', [ ] ])
for x in all_ethnicities:
    flag = True
    for y in all_ethnicities_super:
        if x in y[1]:
            flag = False
            break
    if flag:
        all_ethnicities_super[8][1].append(x)
all_ethnicities_super[8]            

['other / unknown / not specified',
 ['UNKNOWN/NOT SPECIFIED',
  'OTHER',
  'UNABLE TO OBTAIN',
  'PATIENT DECLINED TO ANSWER',
  'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE',
  'AMERICAN INDIAN/ALASKA NATIVE',
  'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER']]

In [27]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: MARITAL_SUPER
# Previous cells: ETHNICITIES_&_MARITAL
# Next cells: ONEHOT_DATA_LENGTH
# --------------------------------------------------------------------------------------------------------------

# unifies certain marital values
all_marital_super = []
all_marital_super.append([ 'divorced', [ 'DIVORCED' ] ])
all_marital_super.append([ 'single', [ 'SINGLE' ] ])
all_marital_super.append([ 'married', [ 'MARRIED', 'LIFE PARTNER' ] ])
all_marital_super.append([ 'widowed', [ 'WIDOWED' ] ])
all_marital_super.append([ 'seperated', [ 'SEPARATED' ] ])
all_marital_super.append([ 'other / not specified', [ 'UNKNOWN (DEFAULT)', '' ] ])

In [28]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ONEHOT_DATA_LENGTH
# Previous cells: MARITAL_SUPER
# Next cells: INSTANCE_LABELS_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

# determines one-hot array length of sex, ethnicity and marital data
onehot_data_length = 2 + len(all_ethnicities_super) + len(all_marital_super) # +2 is male/female
onehot_data_length

17

In [29]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: INSTANCE_LABELS_CATEGORICAL
# Previous cells: ETHNICITIES_SUPER, MARITAL_SUPER
# Next cells: NORMALIZED_DATA_CATEGORICAL, DATA_LEN
# --------------------------------------------------------------------------------------------------------------

# creates an instance labels list, this is mainly for our clarity's sake
instance_labels_categorical = ['subject_id', 'hadm_id', 'survived', ['age', 'length of stay'], ['male','female']]
all_icd_categories = []
for x in all_ethnicities_super:
    instance_labels_categorical[4].append(x[0])
for x in all_marital_super:
    instance_labels_categorical[4].append(x[0])
categories = ['V','E','100','200','250','285','300','370','400','500','550','600','650','700','720','750','770','790','900']
for x in categories:
    c = categorize(x)
    instance_labels_categorical[4].append(c)
    all_icd_categories.append(c)

In [30]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ICDS_3DIGIT
# Previous cells: FINAL_DATA_SORTED
# Next cells: INSTANCE_LABELS_3DIGIT, NORMALIZED_DATA_3DIGIT
# --------------------------------------------------------------------------------------------------------------

# creates the multi-hot list of ICDs in 3-digit format
icdlist_3digit = []
for x in final_data:
    for x9 in x[9]:
        temp = digitize3(x9)      
        if temp not in icdlist_3digit:
            icdlist_3digit.append(temp)
icdlist_3digit = icdSort(icdlist_3digit)
len(icdlist_3digit)

909

In [31]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ICDS_4DIGIT
# Previous cells: FINAL_DATA_SORTED
# Next cells: INSTANCE_LABELS_4DIGIT, NORMALIZED_DATA_4DIGIT
# --------------------------------------------------------------------------------------------------------------

# creates the multi-hot list of ICDs in 4-digit format
icdlist_4digit = []
for x in final_data:
    for x9 in x[9]:
        if x9 not in icdlist_4digit:
            icdlist_4digit.append(x9)
icdlist_4digit = icdSort(icdlist_4digit)
len(icdlist_4digit)

2983

In [32]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: INSTANCE_LABELS_3DIGIT
# Previous cells: ETHNICITIES_SUPER, MARITAL_SUPER, ICDS_3DIGIT
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

instance_labels_3digit = ['subject_id', 'hadm_id', 'survived', ['age', 'length of stay'], ['male','female']]
for x in all_ethnicities_super:
    instance_labels_3digit[4].append(x[0])
for x in all_marital_super:
    instance_labels_3digit[4].append(x[0])
for x in icdlist_3digit:
    instance_labels_3digit[4].append(x)

In [33]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: INSTANCE_LABELS_4DIGIT
# Previous cells: ETHNICITIES_SUPER, MARITAL_SUPER, ICDS_4DIGIT
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

instance_labels_4digit = ['subject_id', 'hadm_id', 'survived', ['age', 'length of stay'], ['male','female']]
for x in all_ethnicities_super:
    instance_labels_4digit[4].append(x[0])
for x in all_marital_super:
    instance_labels_4digit[4].append(x[0])
for x in icdlist_4digit:
    instance_labels_4digit[4].append(x)

In [34]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: NORMALIZED_DATA_CATEGORICAL
# Previous cells: INSTANCE_LABELS_CATEGORICAL
# Next cells: IO_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

# normalizes all data from raw to be ready to be input into the neural network, in the categorical data aggregation,
# that means mean and variance normalization for age and leangth of stay,
# one-hot arrays for ethnicity and marital status,
# and the categorical icd multi-hot list
normalized_data_categorical = []
for x in final_data:
    instance = []
    instance.append(x[0])
    instance.append(x[1])
    if x[7]:
        instance.append(1)
    else:
        instance.append(0)
    temp = []
    temp.append((x[4]-mean_age)/variance_age)
    temp.append((x[8]-mean_los)/variance_los)
    instance.append(temp)
    temp = []
    if x[3] == 'M':
        temp.extend([1,0])
    else:
        temp.extend([0,1])
    for e in all_ethnicities_super:
        if x[5] in e[1]:
            temp.append(1)
        else:
            temp.append(0)
    for m in all_marital_super:
        if x[6] in m[1]:
            temp.append(1)
        else:
            temp.append(0)
    for c in all_icd_categories:
        flag = 0
        for icd in x[9]:
            if categorize(icd) == c:
                flag = 1
                break
        temp.append(flag)
    instance.append(temp)
    normalized_data_categorical.append(instance)
normalized_data_categorical[:5]

[['17',
  '194023',
  1,
  [-1.0533382442257435, -0.5657337164652282],
  [0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0]],
 ['17',
  '161087',
  1,
  [-0.9879086085081608, -0.5657337164652282],
  [0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0]],
 ['21',
  '109451',
  1,
  [1.5638471844775625, 0.18348988144102577],
  [1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0]],
 ['21',
  '111970',
  0,
  [1.629276820195145, -0.23274545072911534],
  [1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [35]:
instance_labels_categorical

['subject_id',
 'hadm_id',
 'survived',
 ['age', 'length of stay'],
 ['male',
  'female',
  'white - western',
  'black - african american',
  'hispanic / latino',
  'white - eastern',
  'asian',
  'black - non american',
  'middle eastern',
  'multirace',
  'other / unknown / not specified',
  'divorced',
  'single',
  'married',
  'widowed',
  'seperated',
  'other / not specified',
  'V01 - 99',
  'E001 - E999',
  '1 - 139',
  '140 - 239',
  '240 - 279',
  '280 - 289',
  '290 - 319',
  '320 - 389',
  '390 - 459',
  '460 - 519',
  '520 - 579',
  '580 - 629',
  '630 - 679',
  '680 - 709',
  '710 - 739',
  '740 - 759',
  '760 - 779',
  '780 - 799',
  '800 - 999']]

In [36]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: NORMALIZED_DATA_3DIGIT
# Previous cells: ETHNICITIES_SUPER, MARITAL_SUPER, ICDS_3DIGIT
# Next cells: IO_3DIGIT
# --------------------------------------------------------------------------------------------------------------

# normalizes all data from raw to be ready to be input into the neural network, in the 3-digit data aggregation,
# that means mean and variance normalization for age and leangth of stay,
# one-hot arrays for ethnicity and marital status,
# and the 3-digit icd multi-hot list
normalized_data_3digit = []
for x in final_data:
    instance = []
    instance.append(x[0])
    instance.append(x[1])
    if x[7]:
        instance.append(1)
    else:
        instance.append(0)
    temp = []
    temp.append((x[4]-mean_age)/variance_age)
    temp.append((x[8]-mean_los)/variance_los)
    instance.append(temp)
    temp = []
    if x[3] == 'M':
        temp.extend([1,0])
    else:
        temp.extend([0,1])
    for e in all_ethnicities_super:
        if x[5] in e[1]:
            temp.append(1)
        else:
            temp.append(0)
    for m in all_marital_super:
        if x[6] in m[1]:
            temp.append(1)
        else:
            temp.append(0)
    for li in icdlist_3digit:
        flag = 0
        for icd in x[9]:
            if digitize3(icd) == li:
                flag = 1
                break
        temp.append(flag)
    instance.append(temp)
    normalized_data_3digit.append(instance)
normalized_data_3digit[:5]

[['17',
  '194023',
  1,
  [-1.0533382442257435, -0.5657337164652282],
  [0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,

In [37]:
instance_labels_3digit

['subject_id',
 'hadm_id',
 'survived',
 ['age', 'length of stay'],
 ['male',
  'female',
  'white - western',
  'black - african american',
  'hispanic / latino',
  'white - eastern',
  'asian',
  'black - non american',
  'middle eastern',
  'multirace',
  'other / unknown / not specified',
  'divorced',
  'single',
  'married',
  'widowed',
  'seperated',
  'other / not specified',
  'V01',
  'V02',
  'V03',
  'V04',
  'V05',
  'V06',
  'V07',
  'V08',
  'V09',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V26',
  'V27',
  'V40',
  'V42',
  'V43',
  'V44',
  'V45',
  'V46',
  'V48',
  'V49',
  'V50',
  'V51',
  'V53',
  'V54',
  'V55',
  'V56',
  'V58',
  'V59',
  'V60',
  'V61',
  'V62',
  'V63',
  'V64',
  'V65',
  'V66',
  'V67',
  'V69',
  'V70',
  'V71',
  'V72',
  'V74',
  'V76',
  'V84',
  'V85',
  'V86',
  'V87',
  'V88',
  'V90',
  'E000',
  'E001',
  'E805',
  'E812',
  'E813',
  'E814',
  'E815',
  'E816',
  'E817',
  'E818',
  'E819'

In [39]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: NORMALIZED_DATA_4DIGIT
# Previous cells: ETHNICITIES_SUPER, MARITAL_SUPER, ICDS_4DIGIT
# Next cells: IO_4DIGIT
# --------------------------------------------------------------------------------------------------------------

# normalizes all data from raw to be ready to be input into the neural network, in the 4-digit data aggregation,
# that means mean and variance normalization for age and leangth of stay,
# one-hot arrays for ethnicity and marital status,
# and the 4-digit icd multi-hot list
normalized_data_4digit = []
for x in final_data:
    instance = []
    instance.append(x[0])
    instance.append(x[1])
    if x[7]:
        instance.append(1)
    else:
        instance.append(0)
    temp = []
    temp.append((x[4]-mean_age)/variance_age)
    temp.append((x[8]-mean_los)/variance_los)
    instance.append(temp)
    temp = []
    if x[3] == 'M':
        temp.extend([1,0])
    else:
        temp.extend([0,1])
    for e in all_ethnicities_super:
        if x[5] in e[1]:
            temp.append(1)
        else:
            temp.append(0)
    for m in all_marital_super:
        if x[6] in m[1]:
            temp.append(1)
        else:
            temp.append(0)
    for li in icdlist_4digit:
        flag = 0
        for icd in x[9]:
            if icd == li:
                flag = 1
                break
        temp.append(flag)
    instance.append(temp)
    normalized_data_4digit.append(instance)
normalized_data_4digit[:5]

KeyboardInterrupt: 

In [ ]:
instance_labels_4digit

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: INSTANCE_INFORMATION
# Previous cells: NORMALIZED_DATA_CATEGORICAL
# Next cells: WRITE_ALL
# --------------------------------------------------------------------------------------------------------------

instance_information = [[[0 for i in range(3)] for j in range(42)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(instance_information)):
    cnt = 1
    for curr in normalized_data_categorical[big_cnt+1:]:
        big_cnt += 1
        visits = int(myUnsortedSearch(over1counters,0,prev[0])[0][1])
        instance_information[i][-cnt][0] = visits-cnt+1
        instance_information[i][-cnt][1] = prev[0]
        instance_information[i][-cnt][2] = prev[1]
        if prev[0] != curr[0]:
            prev = curr
            break
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: FLOAT_INPUTS
# Previous cells: NORMALIZED_DATA_CATEGORICAL
# Next cells: WRITE_FLOAT_INPUTS
# --------------------------------------------------------------------------------------------------------------

# 41 because we only care about the inputs, last one is discarded
float_inputs = [[[0 for i in range(2)] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(float_inputs)):
    cnt = 1
    for curr in normalized_data_categorical[big_cnt+1:]:
        if cnt == 42:
            break
        big_cnt += 1
        if prev[0] != curr[0]:
            prev = curr
            break
        else:
            float_inputs[i][-cnt][0] = prev[3][0]
            float_inputs[i][-cnt][1] = prev[3][1]
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ONEHOT_INPUTS
# Previous cells: NORMALIZED_DATA_3DIGIT
# Next cells: WRITE_ONEHOT_INPUTS
# --------------------------------------------------------------------------------------------------------------

# 41 because we only care about the inputs, last one is discarded
onehot_inputs = [[[0 for i in range(onehot_data_length)] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(float_inputs)):
    cnt = 1
    for curr in normalized_data_categorical[big_cnt+1:]:
        if cnt == 42:
            break
        big_cnt += 1
        if prev[0] != curr[0]:
            prev = curr
            break
        else:
            onehot_inputs[i][-cnt] = prev[4][:onehot_data_length]
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: IO_CATEGORICAL
# Previous cells: NORMALIZED_DATA_CATEGORICAL
# Next cells: WRITE_INPUTS_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

icd_inputs_categorical = [[[0 for i in range(len(all_icd_categories))] for j in range(41)] for k in range(len(final_patients))]
icd_outputs_categorical = [[0 for i in range(len(all_icd_categories))] for j in range(len(final_patients))]
big_cnt = 0
outputs_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(icd_inputs_categorical)):
    cnt = 1
    for curr in normalized_data_categorical[big_cnt+1:]:
        big_cnt += 1
        if prev[0] != curr[0]:
            icd_outputs_categorical[outputs_cnt] = prev[4][onehot_data_length:]
            outputs_cnt += 1
            prev = curr
            break
        else:
            icd_inputs_categorical[i][-cnt] = prev[4][onehot_data_length:]
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: IO_3DIGIT
# Previous cells: NORMALIZED_DATA_3DIGIT
# Next cells: WRITE_INPUTS_3DIGIT
# --------------------------------------------------------------------------------------------------------------

icd_inputs_3digit = [[[0 for i in range(len(icdlist_3digit))] for j in range(41)] for k in range(len(final_patients))]
icd_outputs_3digit = [[0 for i in range(len(icdlist_3digit))] for j in range(len(final_patients))]
big_cnt = 0
outputs_cnt = 0
prev = normalized_data_3digit[0]
for i in range(len(icd_inputs_3digit)):
    cnt = 1
    for curr in normalized_data_3digit[big_cnt+1:]:
        big_cnt += 1
        if prev[0] != curr[0]:
            icd_outputs_3digit[outputs_cnt] = prev[4][onehot_data_length:]
            outputs_cnt += 1
            prev = curr
            break
        else:
            icd_inputs_3digit[i][-cnt] = prev[4][onehot_data_length:] # +1 because one goes to outputs
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: IO_4DIGIT
# Previous cells: NORMALIZED_DATA_4DIGIT
# Next cells: WRITE_4DIGIT
# --------------------------------------------------------------------------------------------------------------

icd_inputs_4digit = [[[0 for i in range(len(icdlist_4digit))] for j in range(41)] for k in range(len(final_patients))]
icd_outputs_4digit = [[0 for i in range(len(icdlist_4digit))] for j in range(len(final_patients))]
big_cnt = 0
outputs_cnt = 0
prev = normalized_data_4digit[0]
for i in range(len(icd_inputs_4digit)):
    cnt = 1
    for curr in normalized_data_4digit[big_cnt+1:]:
        big_cnt += 1
        if prev[0] != curr[0]:
            icd_outputs_4digit[outputs_cnt] = prev[4][onehot_data_length:]
            outputs_cnt += 1
            prev = curr
            break
        else:
            icd_inputs_4digit[i][-cnt] = prev[4][onehot_data_length:] # +1 because one goes to outputs
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: WRITE_INSTANCE_INFORMATION
# Previous cells: INSTANCE_INFORMATION
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - inputs - information.txt", "w")
for x in instance_information:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: WRITE_FLOAT_INPUTS
# Previous cells: FLOAT_INPUTS
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - inputs - float.txt", "w")
for x in float_inputs:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: WRITE_ONEHOT_INPUTS
# Previous cells: ONEHOT_INPUTS
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - inputs - onehot.txt", "w")
for x in onehot_inputs:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: WRITE_CATEGORICAL
# Previous cells: IO_CATEGORICAL
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - outputs - categorical.txt", "w")
for x in icd_outputs_categorical:
    f.write(str(x) + '\n')
f.close()
f = open("training data - inputs - categorical.txt", "w")
for x in icd_inputs_categorical:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: WRITE_3DIGIT
# Previous cells: IO_3DIGIT
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - outputs - 3digit.txt", "w")
for x in icd_outputs_3digit:
    f.write(str(x) + '\n')
f.close()
f = open("training data - inputs - 3digit.txt", "w")
for x in icd_inputs_3digit:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: WRITE_4DIGIT
# Previous cells: IO_4DIGIT
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - outputs - 4digit.txt", "w")
for x in icd_outputs_4digit:
    f.write(str(x) + '\n')
f.close()
f = open("training data - inputs - 4digit.txt", "w")
for x in icd_inputs_4digit:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

PADDING AFTER FILE WRITING

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: FLOAT_INPUTS_PADDING_AFTER
# Previous cells: NORMALIZED_DATA_CATEGORICAL
# Next cells: WRITE_INPUTS_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

float_inputs_padding_after = [[[0 for i in range(2)] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(float_inputs_padding_after)):
    cnt = 2
    for curr in normalized_data_categorical[big_cnt+1:]:
        if cnt == 41:
            break
        big_cnt += 1
        if prev[0] == curr[0]:
            visits = int(myUnsortedSearch(over1counters,0,prev[0])[0][1])
            float_inputs_padding_after[i][visits-cnt][0] = prev[3][0]
            float_inputs_padding_after[i][visits-cnt][1] = prev[3][1]
        else:
            prev = curr
            break
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ONEHOT_INPUTS_PADDING_AFTER
# Previous cells: NORMALIZED_DATA_CATEGORICAL
# Next cells: WRITE_INPUTS_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

onehot_inputs_padding_after = [[[0 for i in range(onehot_data_length)] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(onehot_inputs_padding_after)):
    cnt = 2
    for curr in normalized_data_categorical[big_cnt+1:]:
        if cnt == 41:
            break
        big_cnt += 1
        if prev[0] == curr[0]:
            visits = int(myUnsortedSearch(over1counters,0,prev[0])[0][1])
            onehot_inputs_padding_after[i][visits-cnt] = prev[4][:onehot_data_length]
        else:
            prev = curr
            break
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: IO_CATEGORICAL_PADDING_AFTER
# Previous cells: NORMALIZED_DATA_CATEGORICAL
# Next cells: WRITE_INPUTS_CATEGORICAL
# --------------------------------------------------------------------------------------------------------------

icd_inputs_categorical_padding_after = [[[0 for i in range(len(all_icd_categories))] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_categorical[0]
for i in range(len(icd_inputs_categorical_padding_after)):
    cnt = 2
    for curr in normalized_data_categorical[big_cnt+1:]:
        if cnt == 41:
            break
        big_cnt += 1
        if prev[0] == curr[0]:
            visits = int(myUnsortedSearch(over1counters,0,prev[0])[0][1])
            icd_inputs_categorical_padding_after[i][visits-cnt] = prev[4][onehot_data_length:]
        else:
            prev = curr
            break
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: IO_3DIGIT_PADDING_AFTER
# Previous cells: NORMALIZED_DATA_3DIGIT
# Next cells: WRITE_INPUTS_3DIGIT
# --------------------------------------------------------------------------------------------------------------

icd_inputs_3digit_padding_after = [[[0 for i in range(len(icdlist_3digit))] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_3digit[0]
for i in range(len(icd_inputs_3digit_padding_after)):
    cnt = 2
    for curr in normalized_data_3digit[big_cnt+1:]:
        if cnt == 41:
            break
        big_cnt += 1
        if prev[0] == curr[0]:
            visits = int(myUnsortedSearch(over1counters,0,prev[0])[0][1])
            icd_inputs_3digit_padding_after[i][visits-cnt] = prev[4][onehot_data_length:]
        else:
            prev = curr
            break
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: IO_4DIGIT_PADDING_AFTER
# Previous cells: NORMALIZED_DATA_4DIGIT
# Next cells: WRITE_INPUTS_4DIGIT
# --------------------------------------------------------------------------------------------------------------

icd_inputs_4digit_padding_after = [[[0 for i in range(len(icdlist_4digit))] for j in range(41)] for k in range(len(final_patients))]
big_cnt = 0
prev = normalized_data_4digit[0]
for i in range(len(icd_inputs_4digit_padding_after)):
    cnt = 2
    for curr in normalized_data_4digit[big_cnt+1:]:
        if cnt == 41:
            break
        big_cnt += 1
        if prev[0] == curr[0]:
            visits = int(myUnsortedSearch(over1counters,0,prev[0])[0][1])
            icd_inputs_4digit_padding_after[i][visits-cnt] = prev[4][onehot_data_length:]
        else:
            prev = curr
            break
        cnt += 1
        prev = curr

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# Cell ID: ICD_INPUTS_PADDING_AFTER
# Previous cells: IO_4DIGIT
# Next cells: none
# --------------------------------------------------------------------------------------------------------------

f = open("training data - inputs - float - padding after.txt", "w")
for x in float_inputs_padding_after:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

f = open("training data - inputs - onehot - padding after.txt", "w")
for x in onehot_inputs_padding_after:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

f = open("training data - inputs - categorical - padding after.txt", "w")
for x in icd_inputs_categorical_padding_after:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

f = open("training data - inputs - 3digit - padding after.txt", "w")
for x in icd_inputs_3digit_padding_after:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

f = open("training data - inputs - 4digit - padding after.txt", "w")
for x in icd_inputs_4digit_padding_after:
    for x2 in x:
        f.write(str(x2) + '\n')
    f.write("*new timeseries*\n")
f.close()

In [ ]:
#playsound('TOASTY!.mp3')

# ALL BELOW IS EXTRA